In [61]:
!pip install transformers PyPDF2 langchain



In [79]:
!pip install transformers PyPDF2
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from PyPDF2 import PdfReader
from google.colab import files
from PyPDF2 import PdfReader
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
import os
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [80]:
# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [81]:
def answer_question(context, question):
    """ Generate an answer to a question based on the provided context. """
    input_text = f"question: {question} context: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    outputs = model.generate(input_ids)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [82]:
def load_and_process_pdf(file_path):
    """ Load and process PDF file, extracting text. """
    reader = PdfReader(file_path)
    full_text = " ".join([page.extract_text() for page in reader.pages if page.extract_text() is not None])
    return full_text

In [83]:
def summarize_conversation(conversation_history):
    """ Summarize the conversation history. """
    text = " ".join(conversation_history)
    input_ids = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True).input_ids
    outputs = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [85]:
# Get your API keys from Openai, you will need to create an account.
import os
os.environ["OPENAI_API_KEY"] = "api_key"

In [86]:
from google.colab import drive
drive.mount('/content/drive')

# Now you can access files in your Google Drive using paths like '/content/drive/My Drive/path_to_your_file.pdf'
file_path = '/content/drive/My Drive/dmv.pdf'
pdf_reader = PdfReader(file_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [87]:
from PyPDF2 import PdfReader

# Initialize an empty string to hold all the text
text = ''

# Loop through each page in the PDF
for i, page in enumerate(pdf_reader.pages):
    page_text = page.extract_text()  # Extract text from each page
    if page_text:  # If text was found, append it to the text variable
        text += page_text

# # Now 'text' contains all the text extracted from the PDF
# print(text)  # Optionally print the text to check


In [88]:
text_splitter = RecursiveCharacterTextSplitter(
chunk_size = 512,
chunk_overlap  = 32,
length_function = len,
)
texts = text_splitter.split_text(text)

In [89]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [90]:
!pip install faiss-gpu


In [91]:
import faiss  # Importing after installation to ensure it loads correctly
from langchain_community.vectorstores import FAISS  # Ensure this import is correct based on your use of langchain

# Example usage (assuming you have embeddings and texts prepared)
docsearch = FAISS.from_texts(texts, embeddings)


In [92]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")


In [93]:
query = "what is the file about?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The file is about filing a complaint against a law enforcement agency and the rights that individuals have when doing so. It also mentions the requirement to file a report with DMV in the event of a traffic accident, regardless of who caused it. Additionally, the file mentions that the DMV owns the copyright to the work and outlines the illegal activities related to copying or distributing the handbook.'

In [94]:
query = "What does the section 'Driving school' talk about?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" The section talks about selecting a driving school and provides information for applying for a driver's license. It also mentions the requirements for practicing driving and taking a behind-the-wheel drive test."

In [95]:
query = "What does section 6 talk about?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Section 6 talks about navigating the roads.'